# **Introducción**
Este script tiene como objetivo extraer datos de recetas de comida desde la API pública de TheMealDB, transformarlos y cargarlos en una tabla de Amazon Redshift. La selección de variables se realizó para capturar la información más relevante y completa sobre cada receta

# **Instalacion de librerías**

In [ ]:

!pip install python-dotenv
!pip install requests psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.8 MB/s eta 0:00:00


In [ ]:
import os
# Librería estándar de Python para interactuar con el sistema operativo, manejar rutas y variables de entorno.

from dotenv import load_dotenv
# Librería para cargar variables de entorno desde un archivo .env, útil para gestionar credenciales de forma segura.

import requests
# Librería para hacer solicitudes HTTP en Python, ideal para interactuar con APIs y obtener datos en formato JSON.

import psycopg2
# Librería para conectarse y ejecutar consultas en bases de datos PostgreSQL y Amazon Redshift.

import json
# Librería estándar de Python para trabajar con datos en formato JSON, permitiendo la serialización y deserialización de datos.


# **Se crea el archivo .env**

In [ ]:
with open('.env', 'w') as f:
    f.write("""
DB_HOST=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com
DB_PORT=5439
DB_USER=bf_miguel_coderhouse
DB_PASS=HW767tUWkj
DB_NAME=dev
    """)

# Verificacion del archivo
!cat .env



DB_HOST=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com
DB_PORT=5439
DB_USER=bf_miguel_coderhouse
DB_PASS=HW767tUWkj
DB_NAME=dev
    

# **Se cargan las variables de entorno**

In [ ]:
load_dotenv('.env')

# Verificar que las variables se han cargado correctamente
print(os.getenv('DB_HOST'))
print(os.getenv('DB_USER'))

data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com
bf_miguel_coderhouse


# **Se realiza la configuración de la API**

In [ ]:
api_url = "https://www.themealdb.com/api/json/v1/1/search.php?s=Arrabiata"
response = requests.get(api_url)
data = response.json()

# **Se extraen y transforman los datos**

Extraemos los campos relevantes de cada receta y los organizamos en una lista de tuplas (transformed_data). Los campos seleccionados son:

- idMeal: Identificador único de la receta.
- strMeal: Nombre de la receta.
- strCategory: Categoría de la receta (e.g., vegetariana).
- strArea: Región de origen de la receta.
- strInstructions: Instrucciones para preparar la receta.
- strMealThumb: URL de la imagen de la receta.
- strTags: Etiquetas asociadas a la receta.
- strYoutube: URL del video de YouTube de la receta.

In [ ]:
meals = data['meals']
transformed_data = []
for meal in meals:
    transformed_data.append((
        meal['idMeal'],
        meal['strMeal'],
        meal['strCategory'],
        meal['strArea'],
        meal['strInstructions'],
        meal['strMealThumb'],
        meal['strTags'],
        meal['strYoutube']
    ))

# **Se realiza la configuración de la conexión a Redshift**

In [ ]:

conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASS'),
    dbname='data-engineer-database'
)
cur = conn.cursor()

try:
    # Crear la tabla en Redshift con tipos de datos ajustados
    create_table_query = """
    CREATE TABLE IF NOT EXISTS meal_data (
        idMeal VARCHAR(50),
        strMeal TEXT,
        strCategory TEXT,
        strArea TEXT,
        strInstructions TEXT,
        strMealThumb TEXT,
        strTags TEXT,
        strYoutube TEXT,
        ingestion_time TIMESTAMP DEFAULT GETDATE()
    );
    """
    cur.execute(create_table_query)
    conn.commit()

    # Insertar los datos en la tabla
    insert_query = """
    INSERT INTO meal_data (idMeal, strMeal, strCategory, strArea, strInstructions, strMealThumb, strTags, strYoutube)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    for record in transformed_data:
        try:
            cur.execute(insert_query, record)
        except Exception as e:
            print(f"Error inserting record {record}: {e}")
            conn.rollback()  # Abortar la transacción en caso de error

    conn.commit()  # Confirmar todas las inserciones exitosas

except Exception as e:
    print(f"Error creating table or inserting data: {e}")
    conn.rollback()
finally:
    # Cerrar la conexión
    cur.close()
    conn.close()

print("Proceso completado")

Error inserting record ('52771', 'Spicy Arrabiata Penne', 'Vegetarian', 'Italian', 'Bring a large pot of water to a boil. Add kosher salt to the boiling water, then add the pasta. Cook according to the package instructions, about 9 minutes.\r\nIn a large skillet over medium-high heat, add the olive oil and heat until the oil starts to shimmer. Add the garlic and cook, stirring, until fragrant, 1 to 2 minutes. Add the chopped tomatoes, red chile flakes, Italian seasoning and salt and pepper to taste. Bring to a boil and cook for 5 minutes. Remove from the heat and add the chopped basil.\r\nDrain the pasta and add it to the sauce. Garnish with Parmigiano-Reggiano flakes and more basil and serve warm.', 'https://www.themealdb.com/images/media/meals/ustsqw1468250014.jpg', 'Pasta,Curry', 'https://www.youtube.com/watch?v=1IszT_guI08'): value too long for type character varying(256)

Proceso completado
